<a href="https://colab.research.google.com/github/zulfiqaralimir/AI-Powered-SocialMediaPostOptimizer_LLM/blob/main/Multi_AI_Agents_Chatbots_With_External_Tools_With_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Liberaries

In [ ]:
!pip install langgraph langsmith langchain langchain_groq langchain_community

# For State Management

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

In [ ]:
!pip install arxiv wikipedia

# Working with External Tools
### **Arxiv API Integration**: The wrapper effectively leverages the Arxiv API to perform efficient and accurate searches for scientific papers.

### Researchers can use this wrapper to quickly find and explore relevant scientific literature on their areas of interest, saving time and effort.

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

In [ ]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
# Create a Wikipedia API wrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
# Create a Wikipedia query tool
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
# Invoke the tool with the query
result = wiki_tool.invoke("What is Generative AI?")

# Print the result
print(result)

In [ ]:
# Invoke the tool with the query
result = arxiv_tool.invoke("Attention is all you need")

# Print the result
print(result)

In [ ]:
# Create a list of tools
tools = [wiki_tool]

# Lang Graph Application

In [ ]:
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

## Designing Workflow

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
# Create a new StateGraph instance
graph_builder = StateGraph(State)

## Calling LLM Model

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
groq_api_key = userdata.get("groq_api_key")

**Model = Gemma2-9b-It**

In [ ]:
llm=ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")
llm

# Creating Chatbot


In [ ]:
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Creating Entire Flow of Execution

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

# Compilation

In [ ]:
graph = graph_builder.compile()

In [1]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

# Testing

In [ ]:
user_input = "Hi there!, My name is Zulfiqar"

events = graph.stream(
    {"messages": [("user", user_input)]},
    stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()